In [1]:
#### RUN THE CELL TWICE FOR SOME REASON 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os 
import os
CONDA_PREFIX = os.environ["CONDA_PREFIX"]
LD_LIBRARY_PATH = os.environ["LD_LIBRARY_PATH"]

%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/python3.8/site-packages/tensorrt/
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0,2
%env TF_ENABLE_ONEDNN_OPTS=0
%env TOKENIZERS_PARALLELISM = true

print(os.environ["LD_LIBRARY_PATH"])


env: LD_LIBRARY_PATH=/usr/local/cuda/lib64:/home/coler/anaconda3/envs/covid_proj/lib/:/home/coler/anaconda3/envs/covid_proj/lib/python3.8/site-packages/nvidia/cudnn/lib:/home/coler/anaconda3/envs/covid_proj/lib/python3.8/site-packages/tensorrt/
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0,2
env: TF_ENABLE_ONEDNN_OPTS=0
env: TOKENIZERS_PARALLELISM=true
/usr/local/cuda/lib64:/home/coler/anaconda3/envs/covid_proj/lib/:/home/coler/anaconda3/envs/covid_proj/lib/python3.8/site-packages/nvidia/cudnn/lib:/home/coler/anaconda3/envs/covid_proj/lib/python3.8/site-packages/tensorrt/


In [2]:
### VERIFY TF
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
import getpass
pw = getpass.getpass() #ONLY necessary to clean /tmp folder; see notes below

 ········


In [1]:
import pandas as pd
import requests
from io import StringIO
import numpy as np
import os
import random
from tqdm.notebook import tqdm as prog_bar
from glob import glob

# import ktrain
# from ktrain import text
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as cr

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow as tf
tf.__version__

'2.8.0'

In [3]:
df = pd.read_excel("../covid_proj_data/working_data/working_covid_data.xlsx")
df.label = df.label.apply(lambda lab: "SICK_TALK" if lab == "accept" else "HEALTHY_TALK")

lang_dict = dict(
    spanish = "es",
    english = "en",
    italian = "it",
    polish = "pl",
    german = "de",
    french = "fr"
    )
df["lang"] = df.language.apply(lambda lang: lang_dict[lang])

df.shape

(12576, 8)

In [6]:
!rm -r $EXP_NAME
!mkdir $EXP_NAME

!mkdir $EXP_NAME/$MODELS
!mkdir $EXP_NAME/$CHK
!mkdir $EXP_NAME/$RES

/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
rm: missing operand
Try 'rm --help' for more information.
/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: missing operand
Try 'mkdir --help' for more information.
/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘/’: File exists
/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘/’: File exists
/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘/’: File exists


In [20]:
from train import (
    reset_random_seeds,
    multimax,
    load_best_epoch,
    KTrans
    )
from constants import *
print(MODEL_NAMES[0])

xlm-roberta-large


In [11]:
def down_sample(data,
                ratio=1,
                minor="SICK_TALK",
                major="HEALTHY_TALK"):
    counts = data.loc[:,LABEL_COL].value_counts()
    f = ratio*(counts[minor]/counts[major])
    return pd.concat([data.loc[(data.label == major),:].sample(frac=f),data.loc[(data.label == minor),:]]).sample(frac=1).reset_index(drop=True)
#create subset
dx = pd.concat([down_sample(df.loc[df.language == lang]) for lang in set(df.language)])

#double check
for lang in set(dx.language):
    print(f"{lang}\n----\n{dx.loc[dx.language == lang,:].label.value_counts()}\n ----\n\n")

spanish
----
HEALTHY_TALK    331
SICK_TALK       331
Name: label, dtype: int64
 ----


italian
----
SICK_TALK       249
HEALTHY_TALK    249
Name: label, dtype: int64
 ----


german
----
SICK_TALK       459
HEALTHY_TALK    459
Name: label, dtype: int64
 ----


english
----
HEALTHY_TALK    248
SICK_TALK       248
Name: label, dtype: int64
 ----


french
----
SICK_TALK       647
HEALTHY_TALK    647
Name: label, dtype: int64
 ----


polish
----
SICK_TALK       130
HEALTHY_TALK    130
Name: label, dtype: int64
 ----




In [12]:
lr_dict = {
    # 'xlm-mlm-100-1280': 5.85E-06,
    # 'xlm-mlm-17-1280': 7.10E-06,
    # 'bert-base-multilingual-uncased': 3.27E-06,
    # 'bert-base-multilingual-cased': 3.27E-06,
    # 'xlm-roberta-base': 6.49E-06,
    'xlm-roberta-large': 1.72E-06
    }

In [13]:
def get_save_path():
    return f"{EXP_NAME}/{MODELS}/{MODEL_NAMES[0]}"

def evaluate(boot_train,boot_val,model_name,class_weight=None,lr_dict=None,save=False,del_mod=True,save_best=True):
    #define model and fit
    mod = KTrans(model_name,lr_dict,class_weight=class_weight)
    mod.run_fit(boot_train,boot_val,save_best=save_best)
    if save:
        save_path = get_save_path()
        mod.save_predictor(save_path)
    return mod 


In [14]:
mod = evaluate(dx,None,MODEL_NAMES[0],lr_dict=lr_dict,del_mod=False,save=True,save_best=False)

preprocessing train...
language: es
train sequence lengths:
	mean : 19
	95percentile : 46
	99percentile : 54


Is Multi-Label? False


begin training using onecycle policy with max lr of 1.72e-06...
Epoch 1/12
344/344 [==============================] - 81s 155ms/step - loss: 0.7095 - accuracy: 0.5046
Epoch 2/12
344/344 [==============================] - 52s 151ms/step - loss: 0.6912 - accuracy: 0.5288
Epoch 3/12
344/344 [==============================] - 53s 153ms/step - loss: 0.5752 - accuracy: 0.7384
Epoch 4/12
344/344 [==============================] - 53s 152ms/step - loss: 0.4219 - accuracy: 0.8295
Epoch 5/12
344/344 [==============================] - 53s 152ms/step - loss: 0.3129 - accuracy: 0.8748
Epoch 6/12
344/344 [==============================] - 53s 152ms/step - loss: 0.2710 - accuracy: 0.8941
Epoch 7/12
344/344 [==============================] - 53s 152ms/step - loss: 0.2247 - accuracy: 0.9155
Epoch 8/12
344/344 [==============================] - 52s 151ms/step - loss: 0.2030 - accuracy: 0.9208
Epoch 9/12
344/344 [==============================] - 52s 151ms/step - loss: 0.1743 - ac

In [ ]:
import ktrain
# mod.predictor
predictor = ktrain.load_predictor("./save_model/models/xlm-roberta-large")
# predictor.export_model_to_onnx("./tmp/onnx_model.onnx")
# predictor.save("./tmp/tf_model")
# preds[1].predict_proba(['My computer monitor is blurry.',"this is a test"])
# ?ktrain.text.predictor.TextPredictor.export_model_to_onnx
# preds[1].predict("This is a test")

In [23]:
!nvcc --version

/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


In [2]:
# predictor = ktrain.load_predictor('tmp')
from constants import *
import ktrain
model_path = f'{EXP_NAME}/{MODELS}/{MODEL_NAMES[0]}'
print(model_path)
# from transformers import AutoModel
# model = AutoModel.from_pretrained(model_path, from_tf=True)
# model.save_pretrained(f"{model_path}-pt")

predictor = ktrain.load_predictor(model_path)

./save_model/models/xlm-roberta-large


In [13]:
test = ["Have you seen the latest Game of Thrones?","I feel terrible","I feel pretty good, today"]
predictor.predict_proba(test)

array([[0.9755883 , 0.02441169],
       [0.09592008, 0.90408   ],
       [0.51833284, 0.48166716]], dtype=float32)

In [12]:
predictor.predict(test)

['HEALTHY_TALK', 'SICK_TALK', 'HEALTHY_TALK']

In [91]:
# from sklearn.metrics import classification_report as cr

# f1 = pd.DataFrame()

# for mod,dx in result.groupby(["language"]):
#     dic = {lab:[cr(dx["label"],dx["pred"],output_dict=True)[lab]["f1-score"]] for lab in CATS}
#     dat = pd.DataFrame(dic)
#     dat.index = [mod]#pd.MultiIndex.from_tuples([mod], names=('language'))
#     f1 = f1.append(dat)

# f1.reset_index(drop=False,inplace=True)
# f1["model_name"] = MODEL_NAMES[0]
# f1.rename(columns={"index":"language"},inplace=True)
# f1


In [92]:
# import plotly_express as px

# px.bar(f1,
#        x="model_name",
#        y="SICK_TALK",
#        color="language",
#        barmode="group")


In [93]:
# # f1.pivot(columns=["F1"],values=CATS)
# long_f1 = f1.melt(id_vars=["model_name","language"],value_vars=CATS,value_name="F1",var_name="category")
# px.bar(long_f1,
#        x="model_name",
#        y="F1",
#        facet_row="category",
#        color="language",
#        barmode="group")


In [94]:
# agg_f1 = long_f1.groupby(["model_name","category"]).\
#     agg(mean_F1=("F1","mean")).\
#     reset_index(drop=False,inplace=False).\
#     sort_values(by="mean_F1",ascending=False)

# px.bar(agg_f1,
#        x="model_name",
#        y="mean_F1",
#        facet_row="category",
#        # color="language",
#        barmode="group")
# # agg_f1

In [95]:
# MOD = "xlm-roberta-large"
# result["model_name"] = MOD
# best_res = result.loc[result.model_name == MOD,:]
# for lang,dx in best_res.groupby("language"):
#     print(f"{lang} : \n\n {cr(dx.label,dx.pred)}")

In [96]:
# MOD = "xlm-roberta-large"
# best_res = result.loc[result.model_name == MOD,:]
# print(cr(best_res.label,best_res.pred))

In [97]:
def save_this_file():
    cwd = os.path.abspath("")
    fp = glob(f"{cwd}/*.ipynb")[0]
    fn = fp.split("/")[-1]
    ename = EXP_NAME.replace("./","")
    save_to = f"{cwd}/{ename}/{ename}_{fn}"
    try:
        !cp $fp $save_to
    except ValueError as e:
        print(f"Weird error happened: {e}")

save_this_file()

/bin/bash: /home/coler/anaconda3/envs/covid_proj/lib/libtinfo.so.6: no version information available (required by /bin/bash)
